Notebook containing tools to summarize AND interact with different content types (websites, pdfs, ppts, videos) <br>
The Notebook employs various python packages to extract required contents from URLs <br>
And employs an llm connector to connect to frontier llms (llama, mixtral, gemini, gpt etc - gpt-4o in this case) <br>
There are different sections for eacvh content type and it contains summary and qna - which can be run in Test section under each <br>

# Imports

In [5]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [6]:
from src.llm_connector import create_model_client, chat_with_model, get_model_response

In [7]:
# Website Scraper
from bs4 import BeautifulSoup 

In [ ]:
# PDF reader
# !pip install PyPDF2
from io import BytesIO
from PyPDF2 import PdfReader
from urllib.parse import urlparse

In [9]:
# PPT 
# !pip install python-pptx
from pptx import Presentation

In [150]:
# Youtube
# !pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Test

In [ ]:

response = get_model_response(
    model_name="gpt-4o",
    user_message="What are the different types of clouds?",
    system_message="You are a helpful assistant with an expertise in meterology"
)

if response:
    print(response)

# Websites

## Scraper

In [11]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [12]:
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

## Summarize

In [13]:
system_prompt_website_summary = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [14]:
def get_user_prompt_website_summary(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
def summarize_website(url):
    website = Website(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_summary(website),
        system_message=system_prompt_website_summary
    )
    return response

### Test

In [16]:
def display_website_summary(url):
    summary = summarize_website(url)
    display(Markdown(summary))

In [17]:
display_website_summary("https://cnn.com")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled Breaking News,...
Using OpenAI with model: gpt-4o


CNN's website offers comprehensive coverage of current events, featuring news from categories such as US, World, Politics, Business, Health, Entertainment, Style, Travel, Sports, Science, and Climate. Key stories include geopolitical updates like the Ukraine-Russia and Israel-Hamas conflicts, domestic US politics involving former President Trump, and international business news highlighting mergers and market analyses. The site also discusses social issues, scientific discoveries, and cultural happenings. In addition, CNN provides analyses, opinion pieces, multimedia content, and a section known as CNN Underscored, which offers lifestyle tips and product recommendations. There are also interactive elements such as games, quizzes, and a variety of podcasts covering diverse topics.

## QnA

In [18]:
system_prompt_website_qna = "You are an assistant that analyzes the contents of a website \
and answers questions related to the content of the website, ignoring text that might be navigation related. \
Respond in markdown."

In [19]:
def get_user_prompt_website_qna(website, question):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows \n\n"
    user_prompt += website.text
    user_prompt += f"\n\nPlease answer the following question related to this website: {question}"
    return user_prompt

In [20]:
def website_qna(url, question):
    website = Website(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_qna(website, question),
        system_message=system_prompt_website_qna
    )
    return response

### Test

In [21]:
def display_website_answer(url, question):
    answer = website_qna(url, question)
    display(Markdown(answer))

In [22]:
display_website_answer("https://cnn.com", "What is happening with tesla?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled Breaking News,...
Using OpenAI with model: gpt-4o


The website mentions that Tesla is involved in a settlement related to a lawsuit. Specifically, a lawsuit alleging pervasive workplace harassment faced by a Black worker at Tesla was settled. The article describes the settlement but does not provide additional details about its terms or implications.

# Research Paper
passes the whole paper (better suited for smaller reserach papers/articles) <br>
so for long papers refer to other solution in PDF_LLM repo with vector db and embeddings 

## Pdf Reader

In [69]:
class Paper:
    def __init__(self, url):
        self.url = url
        parsed_url = urlparse(url)
        self.text = ""
        self.title = "No title found"

        if parsed_url.scheme in ('http', 'https'):
            response = requests.get(self.url)
            if response.status_code == 200:
                pdf_bytes = BytesIO(response.content)
                reader = PdfReader(pdf_bytes)
                self._extract(reader)
            else:
                print(f"Failed to fetch PDF. Status code: {response.status_code}")
        elif os.path.isfile(url):
            try:
                with open(url, 'rb') as pdf_file:
                    reader = PdfReader(pdf_file)
                    self._extract(reader)
            except Exception as e:
                print(f"Error reading PDF: {e}")
        else:
            print(f"Invalid file path or URL: {url}")

    def _extract(self, reader):
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        self.text = text
        try:
            self.title = reader.metadata.get("/Title", "No title found") or "No title found"
        except Exception:
            self.title = "No title found"

    def __str__(self):
        return f"Paper(title='{self.title[:1000]}...')"


In [ ]:
paper = Paper("d:/Projects/sandbox/data/energies.pdf")
print(paper)

Paper(title='Strategies for Improving the Resiliency of Distribution Networks in Electric Power Systems during Typhoon and Water-Logging Disasters...')


## Summarize

In [71]:
system_prompt_paper_summary = "You are a research assistant. Your job is to go through the provided article in details, understand the purpose, study the methods and techniques employed, arguments made and the conclusion."

In [72]:
def get_user_prompt_paper_summary(paper):
    user_prompt = f"You are looking at a paper titled {paper.title}"
    user_prompt += "\nPlease provide a short summary of this paper in markdown. \
    After the summary extract the following: \
    1) Title and Author of the research paper. \
    2) Year it was published it \
    3) Objective or aim of the research to specify why the research was conducted \
    4) Background or Introduction to explain the need to conduct this research or any topics the readers must have knowledge about \
    5) Type of research/study/experiment to explain what kind of research it is. \
    6) Methods or methodology to explain what the researchers did to conduct the research \
    7) Results and key findings to explain what the researchers found \
    8) Conclusion tells about the conclusions that can be drawn from this research including limitations and future direction \
    The contents of this paper is as follows:\n\n"
    user_prompt += paper.text
    return user_prompt

In [73]:
def summarize_paper(paper_url):
    paper = Paper(paper_url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_paper_summary(paper),
        system_message=system_prompt_paper_summary
    )
    return response

### Test

In [74]:
def display_paper_summary(paper_url):
    summary = summarize_paper(paper_url)
    display(Markdown(summary))

In [ ]:
display_paper_summary("d:/Projects/sandbox/data/energies.pdf")

Using model: gpt-4o
With system prompt: You are a research assistant. Your job is to go th...
With user prompt: You are looking at a paper titled Strategies for I...
Using OpenAI with model: gpt-4o


# Summary of the Paper

The paper titled "Strategies for Improving the Resiliency of Distribution Networks in Electric Power Systems during Typhoon and Water-Logging Disasters" by Nan Ma et al., explores methods to enhance the resilience of urban electricity distribution networks that are susceptible to damage from typhoons and subsequent flooding. The research mainly focuses on predicting power-grid failure rates in such extreme conditions and suggests dynamic network reconstruction strategies to improve the resilience and flexibility of the grids. The proposed dynamic reconstruction method aims to effectively restore critical power supply post-disaster by optimizing network configuration in real-time, which shows significant improvement in maintaining power supply compared to traditional static methods.

---

1) **Title and Author of the Research Paper:**

   **Title:** Strategies for Improving the Resiliency of Distribution Networks in Electric Power Systems during Typhoon and Water-Logging Disasters

   **Authors:** Nan Ma, Ziwen Xu, Yijun Wang, Guowei Liu, Lisheng Xin, Dafu Liu, Ziyu Liu, Jiaju Shi, and Chen Chen.

2) **Year it was Published:**

   **Year:** 2024

3) **Objective or Aim of the Research:**

   The research aims to study the impact of typhoon and water-logging disasters on urban distribution networks and develop strategies to improve these networks' flexibility and resilience. It proposes a method for predicting failure rates of power grids during such events and suggests strategies for network elasticity improvement post-disaster.

4) **Background or Introduction:**

   Extreme weather events such as typhoons and log rains have become more frequent and pose severe challenges to urban public services such as power supply systems. This is due to potential infrastructure damage leading to power outages and economic losses. The need for improving resiliency and flexibility of power networks in response to these weather-induced challenges is imperative for sustainable urban development.

5) **Type of Research/Study/Experiment:**

   This research employs a methodological study, combining theories from hydrology, climate science, and electrical engineering to model and improve the resiliency of power distribution networks during natural disasters.

6) **Methods or Methodology:**

   The study utilized vulnerability curves to map wind speeds and water depths to predict failure probabilities of distribution network components during typhoons and floods. A multi-objective dynamic reconstruction model was built to adjust network configurations in real-time during disaster evolution. The model was tested on a modified 33-node and a 118-node distribution network with pre-loaded distributed generators.

7) **Results and Key Findings:**

   The findings demonstrated that the dynamic reconstruction strategy proposed effectively improved resiliency of the distribution network post-disaster compared to static reconstruction methods. In particular, it noted a 26% load supply improvement in the 33-node system and a near-total resiliency with close to 95% load supply in the 118-node system—significantly better performance over traditional methods.

8) **Conclusion:**

   The research concluded that the proposed dynamic reconstruction method enhances distribution network resilience under complex wind and flood conditions by maintaining power supply to critical loads more effectively than static strategies. It suggested that further work should refine fault predictions and incorporate integrated disaster response planning using weather information for more efficient resiliency strategies. This study emphasizes the importance of adaptive network configuration for sustainable urban energy supply management during natural disasters. Limitations identified include the assumptions in vulnerability modeling, urging future research for more tailored and precise resilience planning strategies.

## QnA

In [77]:
system_prompt_paper_qna = "You are a research assistant. Your job is to go through the provided article in details, understand the purpose, study the methods and techniques employed, arguments made and the conclusion."

In [78]:
def get_user_prompt_paper_qna(website, question):
    user_prompt = f"You are looking at a paper titled {paper.title}. The contents of this paper is as follows:\n\n"
    user_prompt += paper.text
    user_prompt += f"\n\nPlease answer the following question related to this paper: {question}"
    return user_prompt

In [84]:
def paper_qna(url, question):
    paper = Paper(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_paper_qna(paper, question),
        system_message=system_prompt_paper_qna
    )
    return response

### Test

In [85]:
def display_paper_answer(url, question):
    answer = paper_qna(url, question)
    display(Markdown(answer))

In [ ]:
display_paper_answer("d:/Projects/sandbox/data/energies.pdf", "What are the models used in this paper?")

Using model: gpt-4o
With system prompt: You are a research assistant. Your job is to go th...
With user prompt: You are looking at a paper titled Strategies for I...
Using OpenAI with model: gpt-4o


The paper "Strategies for Improving the Resiliency of Distribution Networks in Electric Power Systems during Typhoon and Water-Logging Disasters" employs several models to address the impact of extreme weather conditions on electric distribution networks and to propose methodologies for improving system resilience. The models used include:

1. **Vulnerability Models**: 
   - **Distribution Network Towers and Lines Vulnerability Model**: These models use vulnerability curves to predict the failure probabilities of towers and lines based on wind speed. The probability is calculated using event probability models and fragility curves representing the likelihood of failure depending on the wind speed.
   - **Substation and Buried Cable Vulnerability Model**: This model calculates the failure probabilities of substations and buried cables in response to water depth from flooding. It utilizes damage curves and vulnerability functions to determine the likelihood of substation failure due to flooding.

2. **Dynamic Microgrid Reconstruction Mathematical Model**:
   - This multi-objective optimization model is designed for reconstructing distribution networks dynamically in response to evolving fault scenarios during typhoon and water-logging disasters. It aims to maximize load supply and minimize switching operations by dynamically adjusting network topology, distributed generation dispatch, and operational strategies.
   - The reconstruction is modeled as a second-order cone programming (SOCP) problem, involving constraints such as power flow balance, generator operation, and maintaining radial topology.

These models collectively address the stochastic nature of disasters and propose strategies for real-time adaptability and resilience enhancement in power distribution networks.

# Presentations

## PPT Reader

In [129]:
class PowerPoint:
    def __init__(self, path):
        self.path = path
        self.slides = []
        self.text = ""
        self.title = "No title found"

        if os.path.isfile(path):
            try:
                presentation = Presentation(path)
                self._extract(presentation)
                self.title = presentation.core_properties.title or self.slides[0] if self.slides else "No title found"
            except Exception as e:
                print(f"Error reading presentation: {e}")
        else:
            print(f"Invalid file path: {path}")

    def _extract(self, presentation):
        all_text = []
        for slide in presentation.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    slide_text.append(shape.text.strip())
            combined_slide = "\n".join(slide_text)
            self.slides.append(combined_slide)
            all_text.append(combined_slide)
        self.text = "\n\n".join(all_text)

    def __str__(self):
        return f"PresentationFile(title='{self.title[:100]}...', slides={len(self.slides)}, text='{self.text[:1000]}...')"


In [130]:
test_presentation_path = "d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx"
presentation = PowerPoint(test_presentation_path)
print(presentation)

PresentationFile(title='PowerPoint Presentation...', slides=10, text='Wildfire Prediction
Proposal

Use cases
Fire occurrence prediction
Area burned and damage prediction
Effect of weather
Severe weather - Lightning
Temperature rise
Effect of human proceses

Market
**To add spending and loss in and due to wildfires in Canada, US and in general
**Existing solutions and limitations

Data
Climate data 
Wildfire data
**Vegetation Data
**Human activity

Model
Proposed - ANN (1)

Model II
Comparison of models for australia - ann still the better choice because of short term memory (5)

Impact
Weather Network - goes with the brand image
Trust - national provider 
Disaster management and urban and commercial planning
Revenue streams
Government - wildfire / Forest department
International UN disaster management
Tracking of carbon footprint etc. for environmental bodies like Environment Canada, UNEP
Private settlements
Wood/paper industries or suppliers
Visibility and smoke level predictions for

## Summarize

In [131]:
system_prompt_presentation_summary = "You are an assistant that analyzes the contents of a presentation \
and provides a short summary in markdown."

In [132]:
def get_user_prompt_presentation_summary(presentation):
    user_prompt = f"You are looking at a presentation titled {presentation.title}"
    user_prompt += "\nThe contents of this presentation is as follows; \
please provide a short summary of this presentation in markdown.\n\n"
    user_prompt += presentation.text
    return user_prompt

In [133]:
def summarize_presentation(url):
    presentation = PowerPoint(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_presentation_summary(presentation),
        system_message=system_prompt_presentation_summary
    )
    return response

### Test

In [134]:
def display_presentation_summary(url):
    summary = summarize_presentation(url)
    display(Markdown(summary))

In [135]:
display_presentation_summary("d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a presentation titled PowerPoin...
Using OpenAI with model: gpt-4o


```markdown
# Wildfire Prediction Proposal

### Overview
This presentation outlines a proposal for predicting wildfires, focusing on models that predict fire occurrence, the area burned, and the effects of weather and human processes. The proposal highlights advancements in the use of Artificial Neural Networks (ANNs) for accurate predictions and explores their potential impact on disaster management.

### Use Cases
- **Fire Occurrence Prediction:** Forecasting the likelihood of wildfire occurrences.
- **Damage Prediction:** Estimating the potential area that could be burned.
- **Weather Impact:** Understanding the role of severe weather, such as lightning and temperature rise.
- **Human Processes:** Analyzing human activities that influence wildfires.

### Market
- **Financial Impact:** Pending data on wildfire-related spending and losses in Canada and the US.
- **Existing Solutions:** Discussion on current solutions and their limitations.

### Data Sources
- Climate data
- Wildfire data
- Proposed inclusion of vegetation data and human activity data.

### Model Proposal
- **ANN Model I:** Proposed model leveraging ANNs.
- **Model Comparison:** Comparison with models used in Australia showing the advantages of ANNs due to their effective short-term memory capabilities.

### Impact & Benefits
- Boosts brand image for weather networks.
- Enhances trust as a national provider in disaster management.
- Influences urban and commercial planning, benefiting government agencies, international bodies, and industries.
- Potential revenue streams include government departments, international disaster management organizations, and private sectors such as wood and paper industries.
- Improves visibility and smoke level predictions for public safety and industry operations.

### Costs
- Estimated cost for proof of concept (POC).
- Data acquisition and storage expenses.
- Training costs associated with model development.

### Implementation
Outlined steps for taking the proposed model from concept to application.

### References / Resources
- Review of ML applications in wildfire sciences - NRC (2020) [Read more](https://cdnsciencepub.com/doi/pdf/10.1139/er-2020-0019#refg23)
- Wildland Fire Suppression and Partnerships by the DOI [Learn more](https://www.doi.gov/wildlandfire/suppression)
- Forest fire risk prediction system, DOI: 10.1016/S0957-4174(03)00095-2
```


## QnA

In [136]:
system_prompt_presentation_qna = "You are an assistant that analyzes the contents of a presentation \
and answers questions related to the content of the presentation, ignoring text that might be navigation related. \
Respond in markdown."

In [137]:
def get_user_prompt_presentation_qna(presentation, question):
    user_prompt = f"You are looking at a presentation titled {presentation.title}"
    user_prompt += "\nThe contents of this presentation is as follows \n\n"
    user_prompt += presentation.text
    user_prompt += f"\n\nPlease answer the following question related to this presentation: {question}"
    return user_prompt

In [138]:
def presentation_qna(url, question):
    presentation = PowerPoint(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_qna(presentation, question),
        system_message=system_prompt_website_qna
    )
    return response

### Test

In [139]:
def display_presentation_answer(url, question):
    answer = presentation_qna(url, question)
    display(Markdown(answer))

In [142]:
display_presentation_answer("d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx", "What is the model and use case proposed?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled PowerPoint Pre...
Using OpenAI with model: gpt-4o


The proposed model for wildfire prediction on the website is an Artificial Neural Network (ANN). The use cases outlined include:

- Fire occurrence prediction
- Area burned and damage prediction
- Effect of weather, specifically severe weather like lightning and temperature rise
- Impact of human processes on wildfire occurrence and behavior

# Videos

## Youtube Transcribe

In [152]:
class YoutubeVideo:
    def __init__(self, url, language='en'):
        """
        Initialize the YoutubeVideo object with the video URL and fetch its transcript.
        """
        self.url = url
        self.video_id = self._extract_video_id(url)
        self.language = language
        self.text = self._get_transcript()
        self.title = f"Video with ID {self.video_id}"  

    def _extract_video_id(self, url):
        """
        Extract the video ID from the given YouTube URL.
        """
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, url)
        if match:
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def _get_transcript(self):
        """
        Fetch the transcript of the video using the YouTubeTranscriptApi.
        """
        try:
            text = YouTubeTranscriptApi.get_transcript(self.video_id, languages=[self.language])
            return " ".join([item['text'] for item in text])
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return None

    def __str__(self):
        """
        String representation of the YoutubeVideo object.
        """
        return f"YoutubeVideo(title='{self.title}', video_id='{self.video_id}', transcript='{self.text[:100]}...')"

## Summarize

In [153]:
system_prompt_video_summary = "You are an assistant that analyzes the contents of a video \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [154]:
def get_user_prompt_video_summary(video):
    user_prompt = f"You are looking at a video titled {video.title}"
    user_prompt += "\nThe contents of this video are transcribed as follows; \
please provide a short summary of this video in markdown. \n\n"
    user_prompt += video.text
    return user_prompt

In [155]:
def summarize_video(url):
    video = YoutubeVideo(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_video_summary(video),
        system_message=system_prompt_video_summary
    )
    return response

### Test

In [156]:
def display_video_summary(url):
    summary = summarize_video(url)
    display(Markdown(summary))

In [157]:
display_video_summary("https://www.youtube.com/watch?v=8HslUzw35mc")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a video titled Video with ID 8H...
Using OpenAI with model: gpt-4o


This video explores homeopathy, a widely debated alternative medicine. It explains key principles, such as "like cures like," where substances causing symptoms are used to treat them, and "potentization," involving extreme dilution to enhance effectiveness. Criticisms arise due to the lack of scientific evidence supporting homeopathy's efficacy beyond placebo effects. Despite this, homeopathy remains popular, partly due to its personalized care and emphasis on empathy—an area modern medicine could potentially learn from. The video also discusses the financial scale of the homeopathy industry and its impact on public health. The video is produced by Kurzgesagt, announcing their relaunch of a German channel with new content.

## QnA

In [160]:
system_prompt_video_qna = "You are an assistant that analyzes the contents of a video \
and answers questions related to the content of the video transcription \
Respond in markdown."

In [161]:
def get_user_prompt_video_qna(video, question):
    user_prompt = f"You are looking at a video titled {video.title}"
    user_prompt += "\nThe contents of this video is as follows \n\n"
    user_prompt += video.text
    user_prompt += f"\n\nPlease answer the following question related to this video: {question}"
    return user_prompt

In [162]:
def video_qna(url, question):
    video = YoutubeVideo(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_video_qna(video, question),
        system_message=system_prompt_video_qna
    )
    return response

### Test

In [163]:
def display_video_answer(url, question):
    answer = video_qna(url, question)
    display(Markdown(answer))

In [ ]:
display_video_answer("https://www.youtube.com/watch?v=8HslUzw35mc", "What is the financial share of homeopathy?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a video titled Video with ID 8H...
Using OpenAI with model: gpt-4o


The video mentions that the global market for homeopathy is expected to reach over $17 billion by 2024. This suggests that homeopathy is a significant industry with substantial financial influence, comparable to other sectors within the pharmaceutical landscape.